# พระอภัยมณี กลอน Generation

## Download dataset and PyTorchLighning

In [1]:
!wget https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
!pip -q install -q lightning

--2025-06-06 16:07:05--  https://github.com/Knight-H/thai-lm/raw/refs/heads/master/data/pra-apai-manee-ch1-50.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Knight-H/thai-lm/refs/heads/master/data/pra-apai-manee-ch1-50.txt [following]
--2025-06-06 16:07:06--  https://raw.githubusercontent.com/Knight-H/thai-lm/refs/heads/master/data/pra-apai-manee-ch1-50.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3231076 (3.1M) [application/octet-stream]
Saving to: ‘pra-apai-manee-ch1-50.txt.5’

pra-apai-manee-ch1- 100%[===================>]   3.08M  --.-KB/s    in 0.05s   

2025-06-06 16:07:07 (58.

## Import libraries

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from datetime import datetime
import os

## Setup

In [3]:
batch_size = 512  # B: จำนวน mini-batch (เทรน 1 ครั้งเห็น data กี่ samples)
seq_len = 256     # T: ความยาวประโยคมากสุด
n_embd = 128      # C: embedding size (ความยาวชุดตัวเลขที่ใช้ทำ auto feature extraction)
n_hidden = 256    # Hidden size for LSTM (ความยาวชุดตัวเลข hidden_state(h) )
n_layers = 2      # Number of LSTM layers (มี LSTM ต่อกันกี่ชั้น)

# กำหนดจำนวนรอบการ train/validate
# รอบที่ว่าคือ iteration (train แค่ 1 batch)
# ไม่ใช่ epoch (train แบบเห็นทั้ง dataset)

eval_iters = 200
max_iters = eval_iters * 10  # train กี่ batch


learning_rate = 1e-3 # learning rate
device = 'cuda' if torch.cuda.is_available() else 'cpu' # ใช้ GPU Nvidia
dropout = 0.2

assert device == "cuda", "This experiment requires a GPU to run."
torch.manual_seed(42)
L.pytorch.seed_everything(42)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

### Load and prepare data

In [4]:
with open('pra-apai-manee-ch1-50.txt', 'r', encoding='utf-8') as f: # อ่านข้อมูล data พระอภัยมณี
    text = f.read()

vocab = sorted(list(set(text))) # คำที่แตกต่างกันทั้งหมดในกลอน
vocab_size = len(vocab) # จำนวนคำที่แตกต่างกัน

stoi = { ch:i for i,ch in enumerate(vocab) } # string to int => เปลี่ยนจากคำเป็น id ของคำ (ต้องทำให้ PyTorch คำนวณได้)
itos = { i:ch for i,ch in enumerate(vocab) } # int to string => แปลงกลับจาก id มาเป็นคำให้คนอ่านได้


encode = lambda s: [stoi[c] for c in s]          # encoder: รับ ประโยคมาแล้วแปลงเป็น ลำดับของ id ของแต่ละคำ
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: แปลงกลับจาก encoder (จาก id เป็น คำ)

data = torch.tensor(encode(text), dtype=torch.long) # แปลงทั้ง dataset จากคำเป็น id ให้หมด

## Dataset


### Splitting data (Train/val)
กรณีนี้ไม่มี test set เพราะเราจะลอง generate กลอนจากที่โมเดลเรียนเลย

In [5]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

### สร้าง PyTorch Dataset และ PyTorch DataLoader

In [6]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - seq_len

    def __getitem__(self, idx):
        return self.data[idx:idx+seq_len], self.data[idx+1:idx+seq_len+1]

train_dataset = TextDataset(train_data, seq_len)
val_dataset = TextDataset(val_data, seq_len)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

## Modeling

### LSTM Model
Model LSTM (RNN family) สำหรับการใช้ทายคำถัดไป

In [7]:
# LSTM Language Model (LSTM เป็น RNN family ตัวนึง => เก่งกว่า RNN)
class LSTMLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=n_embd,
            hidden_size=n_hidden,
            num_layers=n_layers,
            dropout=dropout if n_layers > 1 else 0,
            batch_first=True
        ) # นิยาม model LSTM

        self.dropout = nn.Dropout(dropout)

        # Output layer
        self.lm_head = nn.Linear(n_hidden, vocab_size)

    def forward(self, idx, targets=None, hidden=None):
        B, T = idx.shape

        # Token embeddings
        # แปลงจาก ลำดับ id เป็น ลำดับชุดตัวเลขที่เรียนมา เช่น
        # [3, 4, 1, 4] =>
        # [
        #    [1.5, 3.0, 2.0],     (3)
        #    [-1.0, 2.0, 5.0],    (4)
        #    [-7.0, 5.0, -1.0],   (1)
        #    [-1.0, 2.0, 5.0]     (4)
        # ]
        tok_emb = self.token_embedding_table(idx)  # (B, T, n_embd)

        # LSTM forward pass
        # วิ่งคำนวณ hidden (ht) ตามจำนวน `seq_len` (ในที่นี้เรา set ไว้เป็นความยาว 256)
        # lstm_out คือ hidden อันสุดท้าย => ใช้ predict หาความน่าจะเป็นของคำถัดไป
        lstm_out, hidden = self.lstm(tok_emb, hidden)  # (B, T, n_hidden)

        # Apply dropout
        lstm_out = self.dropout(lstm_out)

        # Generate logits
        # เปลี่ยนจาก hidden state เป็นความ logit ของคำศัพท์แต่ละคำ
        logits = self.lm_head(lstm_out)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits_flat = logits.view(B*T, C)
            targets_flat = targets.view(B*T)
            loss = F.cross_entropy(logits_flat, targets_flat)

        return logits, loss, hidden

    def generate(self, idx, max_new_tokens, temperature=1.0):
        """Generate new tokens"""
        self.eval()
        with torch.no_grad():
            hidden = None
            for _ in range(max_new_tokens):
                # Get the last seq_len tokens (or all if less than seq_len)
                idx_cond = idx[:, -seq_len:] if idx.size(1) > seq_len else idx

                # Forward pass
                logits, _, hidden = self(idx_cond, hidden=hidden)

                # Take the last time step
                logits = logits[:, -1, :] / temperature  # (B, vocab_size)

                # Apply softmax and sample
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

                # Append to sequence
                idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

                # For efficiency, only keep the hidden state from the last seq_len tokens
                if idx.size(1) > seq_len:
                    hidden = None  # Reset hidden state when sequence gets too long

        return idx



### Lightning Module
PyTorch Lightning Module จะช่วยจากการเขียน loop train เองมาเขียนแค่บางส่วน (เดี๋ยวมันจะวนตาม training loop ปกติให้ => เขียนง่ายกว่า PyTorch ปกติมากๆ)

In [8]:
# Lightning Module
class LSTMLMModule(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = LSTMLanguageModel()
        self.save_hyperparameters()

    def forward(self, batch_x, batch_y):
        return self.model(batch_x, batch_y)

    # สร้าง optimizer
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=learning_rate, weight_decay=1e-4)

        # Learning rate scheduler - starts reducing after 3000 steps
        def lr_lambda(step):
            if step < 3000:
                return 1.0  # Keep original learning rate
            else:
                # Exponential decay after 3000 steps
                decay_steps = step - 3000
                return 0.95 ** (decay_steps / 200)  # Decay by 5% every 200 steps

        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",  # Update every step
                "frequency": 1,
            }
        }

    # ตอนเจอ train 1 batch คำนวณอะไรบ้าง => ต้อง return loss เพื่อให้คำนวณ gradient ได้
    def training_step(self, batch, batch_idx):
        xb, yb = batch
        logits, loss, _ = self(xb, yb)
        current_lr = self.trainer.optimizers[0].param_groups[0]['lr']
        self.log("lr", current_lr, prog_bar=True)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    # ตอน validate ทำอะไรบ้าง (ในที่นี้เขียนเหมือน train แต่บาง model validate อาจจะเขียนต่างจากเทรนได้)
    def validation_step(self, val_batch, batch_idx):
        xb, yb = val_batch
        logits, loss, _ = self(xb, yb)
        self.log('val_loss', loss, prog_bar=True)

    def on_train_batch_end(self, outputs, batch, batch_idx):
        metrics = self.trainer.callback_metrics
        if batch_idx % self.trainer.log_every_n_steps == 0:
            now = datetime.now()
            print(f'{now.strftime("%Y-%m-%dT%H:%M:%S")} Step: {batch_idx}/{self.trainer.max_steps} Train Loss: {metrics["train_loss"]:.4f}')

    def on_validation_epoch_end(self):
        metrics = self.trainer.callback_metrics
        print(f'\t\t\tVal Loss: {metrics["val_loss"]:.4f}')

## Train

In [9]:
# Initialize model
model = LSTMLMModule()
print(f"{sum(p.numel() for p in model.parameters())/1e6:.3f} M parameters")

# Training
trainer = L.Trainer(
    deterministic=True,
    accelerator="auto",
    devices="auto",
    logger=False,
    max_steps=max_iters,
    val_check_interval=eval_iters,
    enable_checkpointing=False,
    limit_val_batches=eval_iters,
    log_every_n_steps=500,
    callbacks=[]
)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


0.949 M parameters


### Train model
Train ของ PyTorchLightinng มีความเหมือนกับ scikit-learn มากๆ <br>
1 บรรทัดเท่านั้น จากการเขียน loop เป็นหลายสิบบรรทัด


In [10]:
trainer.fit(model, train_dataloader, val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | LSTMLanguageModel | 948 K  | train
----------------------------------------------------
948 K     Trainable params
0         Non-trainable params
948 K     Total params
3.796     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | LSTMLanguageModel | 948 K  | train
----------------------------------------------------
948 K     Trainable params
0         Non-trainable params
948 K     Total params
3.796     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


			Val Loss: 4.2636


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

			Val Loss: 1.6468


Validation: |          | 0/? [00:00<?, ?it/s]

			Val Loss: 1.6370
2025-06-06T16:20:21 Step: 0/2000 Train Loss: 1.4405


INFO: `Trainer.fit` stopped: `max_steps=2000` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2000` reached.


เมื่อ train เสร็จแล้วให้ save model

In [ ]:
# Save model
trainer.save_checkpoint('../working/lstm_klorn_gen.ckpt')

## ทดลอง Generate กลอน

โหลด model ที่ save ไว้มาใช้ generate กลอน

In [ ]:
best_model = LSTMLMModule.load_from_checkpoint('../working/lstm_klorn_gen.ckpt')

In [13]:
start_context = "๏ อาจารย์หยกแอนดิวสอนเอ็นแอลพี\t" # คำเริ่มต้นประโยค


L.pytorch.seed_everything(42)
context = torch.tensor([encode(start_context)], dtype=torch.long, device=device)
best_model.model.eval()
generated = best_model.model.to(device).generate(context, max_new_tokens=1000, temperature=0.8)
print(decode(generated[0].tolist()))

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


๏ อาจารย์หยกแอนดิวสอนเอ็นแอลพี	สะอื้นหวานเห็นผู้ใหญ่ให้แล้ว
สังเวชน้องร้องไห้ถึงแกล้ง	จะออกนอกความตามให้หายตาย
แม่รักเหมือนบังคมคัตรูรจะฆ่า	สังเพชรนาวัดทัพไม่กลับกลาย
ขี้หลับมาให้เจ้าเมืองผลึก	แล้วชวนทุกข์สุขหยิกสิ้นชีวิต
พระอภัยไม่ทรามตามยิบรับ	อยู่พร้อมพรักสิงห์เป็นหาสนา
จำสำคัญว่ากระไรจะใคร่รู้	แต่รู้หน้าพูดประดำเป็นการสวน
นางฟังคำทำความให้ราคี	จะไปปราศรัยไม่หายสายสมา
เห็นเงาแลดูเรือได้เสื้อถือ	จะบอกน้องเห็นหน้าด้วยอาลัย
พระหัสไชยไปสารพัดจะพลัดพราย	ต่างตรัสภาวนาไปข้าไหน
อันตรองตรึกตรางอยู่ว่าองค์	ก็ควรมาทำมาในอาสัญ
ได้ลงกายากูผู้หญิงชิง	ถึงศอพระองค์ทรงกำปั่นใหญ่
แล้วว่าเฝ้าเช่นนั้นนั้นเหมือนดูฝัง	ยังอยู่พลับพลาฟันเป็นสัญญา
จะต้องรักนั้นด้วยควรจะขอปัญญา	จึงรอราตามเจ้าไม่เข้าใจ
นึกแก้วคนแสนทรงฤทธิ์หน่อย	ถึงลูกเลี่ยงเปลี่ยนเป็นศึกเสียงครึกครื้น
โอ้สงสารพระอภัยใจบุตร	อย่ารู้ว่ายถอยู่ในความหลัง
จึงเสแสร้งแกล้งว่าน่าสงสาร	เชิญออกอาบอกบอกถอนฤทัย ฯ
๏ นางละเวงเกรงใจจะได้คน	เมื่อไม่มีใครบุญผุดพระหน่อนาถ ฯ
๏ พระรักนางอยู่ด้วยบุตรพี่น้อง	เห็นรูปงามความประสงค์จำนงใจ
ทำผ่องเป็นเข้าเข้าแล้วก็ไม่ฟื